# Combining Multiple LoRA Adapters with peft

This notebook shows how to efficiently combine multiple LoRA (Low-Rank Adaptation) adapters using the [peft](https://github.com/huggingface/peft) library to enable multi-task learning in large language models.

LoRA allows adapting pretrained models like LLaMA to new tasks by training only a small set of weights. This enables parameter-efficient fine-tuning.

By combining multiple LoRA adapters, we can leverage knowledge from different specialized adapters to improve performance on downstream tasks that require multiple skills.

In [ ]:
# Install peft library
!pip install peft

In [ ]:
from peft import PeftModel, PeftConfig 
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load base model and tokenizer
model_name = "decapoda-research/llama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load individual LoRA adapters
adapter_names = ["alpaca-lora", "code-lora", "summarize-lora"]
adapters = {name: PeftModel.from_pretrained(model, name) for name in adapter_names}

# Combine adapters using peft's `add_weighted_adapter` method
model.add_weighted_adapter(adapter_names, weights=[1.0, 0.8, 0.5])

# Test the combined model
prompt = """Summarize the following Python code:

def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
"""

input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_new_tokens=50)
print(tokenizer.decode(output[0]))